<a href="https://colab.research.google.com/github/SANGEETHA2000/Solution-to-the-0-1-Knapsack-Problem-based-on-DNA-Encoding-and-Computing-Method/blob/main/Sentiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Dataset of consideration
dataset = '/content/drive/My Drive/sentiments.csv'

In [ ]:
#Connecting to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Reading dataset
import pandas as pd 
data = pd.read_csv(dataset, encoding='ISO-8859-1',names=["label", "id", "date", "flag","user","text"])
data.tail()

,label,id,date,flag,user,text
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599999,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


In [ ]:
#Getting unique values of the column label
data.label.value_counts()

4    800000
0    800000
Name: label, dtype: int64

In [ ]:
#Mapping 4-1 to indicate 1-positive and 0-negative
data["label"] = data["label"].map({4:1,0:0})

In [ ]:
data.label.value_counts()

1    800000
0    800000
Name: label, dtype: int64

In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 51kB 3.9MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.7-cp36-none-any.whl size=30539 sha256=43b0bf8a085bbfefe02c52777ee4e451a5c33df400400f1ebf819e45b507a62b
  Stored in directory: /root/.cache/pip/wheels/e1/f8/e2/b98f79a6b8cc898d8e4102b83acb8a098df7d27500a2bac912
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7304 sha256=521aa93945b00e90a6a9ad25da268a1772b372064dda47c8405b8f1e757a51ee
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19475 sha256=bebe13611c82c89f865650641f98a62589500e587a8a45ce7fdee5eaeaca89df
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.1MB 5.3MB/s 


In [ ]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
    
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

In [ ]:
#Data rows x column values
data.shape

(1600000, 6)

In [ ]:
data = data.sample(20000)
data['text']

566028     @esthertay You can have mine. Same deal here. ...
238522     UGH had the worst sleep ever i have a major ha...
881515     @mileycyrus I'll be in Atlantis in spirit! Hav...
497330     Had to Deleate A guy I really love on my Faceb...
31015      Looking at all the hate twitters about the com...
                                 ...                        
762894     @Oabeywon  when I was 11 all I was doing was p...
1253409    @MDMOLINARI i think the second ones besttt  &a...
294173     At Mikado's with Mary and her family. And i ju...
322806                    Wishing I was going to the plaza! 
1034147    @bryceavary Can't wait to hear the record... I...
Name: text, Length: 20000, dtype: object

In [ ]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [ ]:
import re
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [ ]:
reviews = []
sentences = list(data['text'])
for s in sentences:
    reviews.append(preprocess_text(s))

In [ ]:
print(data.columns.values)

['label' 'id' 'date' 'flag' 'user' 'text']


In [ ]:
#Writing the respective label values as an array
import numpy as np
y = data['label']
y = np.array(list(map(lambda x: 1 if x==1 else 0, y)))
print(y)

[0 0 1 ... 0 0 1]


In [ ]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
#Built in model
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [ ]:
#Converting tokens to IDs
def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

In [ ]:
tokenized_reviews = [tokenize_reviews(review) for review in reviews]

In [ ]:
tokenized_reviews[0]

[14631,
 2696,
 2100,
 2017,
 2064,
 2031,
 3067,
 2168,
 3066,
 2182,
 2228,
 2168,
 3066,
 7249]

In [ ]:
#reviews_with_len = [[review, y[i], len(review)]  for i, review in enumerate(tokenized_reviews)]
max_size = 0
for i in tokenized_reviews:
  if len(i)>max_size:
    max_size = len(i)
reviews_with_len = [[[review[cnt] if cnt < len(review) else 0 for cnt in range(max_size)], y[i], len(review)]
for i, review in enumerate(tokenized_reviews)]

In [ ]:
#Random shuffling of data
import random
random.shuffle(reviews_with_len)

In [ ]:
reviews_with_len.sort(key=lambda x: x[2])

In [ ]:
sorted_reviews_labels = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len]

In [ ]:
#Processing data to suit the input for the model
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32))

In [ ]:
BATCH_SIZE = 32
#Padding with batches
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [ ]:
next(iter(batched_dataset))

(<tf.Tensor: shape=(32, 49), dtype=int32, numpy=
 array([[14978,     0,     0, ...,     0,     0,     0],
        [ 3509,     0,     0, ...,     0,     0,     0],
        [14978,     0,     0, ...,     0,     0,     0],
        ...,
        [ 2009,  7610,     0, ...,     0,     0,     0],
        [ 4005, 12459,     0, ...,     0,     0,     0],
        [ 4199,  2100,     0, ...,     0,     0,     0]], dtype=int32)>,
 <tf.Tensor: shape=(32,), dtype=int32, numpy=
 array([0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0,
        1, 1, 1, 0, 0, 1, 1, 1, 0, 1], dtype=int32)>)

In [ ]:
import math
TOTAL_BATCHES = math.ceil(len(sorted_reviews_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)
#Split into test and train data
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

In [ ]:
#Our model
class TEXT_MODEL(tf.keras.Model): 
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="BERT_Classifier_A"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [ ]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2
DROPOUT_RATE = 0.2
#the number of passes of the entire training dataset
NB_EPOCHS = 5

In [ ]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [ ]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

In [ ]:
text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/5
563/563 [==============================] - 61s 108ms/step - loss: 0.5578 - accuracy: 0.7090
Epoch 2/5
563/563 [==============================] - 61s 109ms/step - loss: 0.3374 - accuracy: 0.8561
Epoch 3/5
563/563 [==============================] - 61s 109ms/step - loss: 0.1114 - accuracy: 0.9603
Epoch 4/5
563/563 [==============================] - 62s 110ms/step - loss: 0.0595 - accuracy: 0.9776
Epoch 5/5
563/563 [==============================] - 61s 109ms/step - loss: 0.0489 - accuracy: 0.9830


In [ ]:
text_model.summary()

Model: "BERT_Classifier_A"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  6104400   
_________________________________________________________________
conv1d (Conv1D)              multiple                  40100     
_________________________________________________________________
conv1d_1 (Conv1D)            multiple                  60100     
_________________________________________________________________
conv1d_2 (Conv1D)            multiple                  80100     
_________________________________________________________________
global_max_pooling1d (Global multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  77056     
_________________________________________________________________
dropout (Dropout)            multiple            

In [ ]:
results = text_model.evaluate(test_data)
print(results)

62/62 [==============================] - 1s 19ms/step - loss: 0.9483 - accuracy: 0.7616
[0.9482903480529785, 0.7615927457809448]


In [ ]:
def get_prediction(sentence):
    tokens = tokenize_reviews(sentence)
    inputs = tf.expand_dims(tokens, 0)

    output = text_model(inputs, training=False)

    sentiment = math.floor(output*2)

    if sentiment == 0:
        print("Ouput of the model: {}\nPredicted sentiment: negative.".format(
            output))
    elif sentiment == 1:
        print("Ouput of the model: {}\nPredicted sentiment: positive.".format(
            output))

In [ ]:
get_prediction("Situation is so sad.")

Ouput of the model: [[0.00027108]]
Predicted sentiment: negative.


In [ ]:
get_prediction("I am so happy.")

Ouput of the model: [[0.9874635]]
Predicted sentiment: positive.
